# Photometric Reductions with Photpipe

This is a very quick tutorial for reducing data on the DARK cluster with photpipe.  The data coming from the IPP has a couple of known issues related to zeropoints and overestimated photometric uncertainties that can be addressed by photpipe.  It can also be helpful to visually inspect the intermediate data products leading to your photometry.

Photpipe takes YSE data (see `YSE_data_coords.ipynb` for accessing YSE data on the DARK cluster) and measures photometry on objects in the field, measures zeropoints using PS1 public catalogs, performs difference imaging, and then performs forced photometry on the weighted-average centroid of the SN location.

There is some photpipe documentation put together by Justin Roberts-Pierel at https://photpipe-docs.readthedocs.io/en/latest/, but it's relatively incomplete for the moment.

In this tutorial, I'll walk through a quick example of how to add a new object to photpipe's list of YSE objects and walk through the full pipeline.  There are a few issues with these steps, and additional developers are always appreciated :).

## Basics of DARK and the Photpipe Environment

To log in to the DARK cluster, SSH to one of the login nodes, e.g.:

```
ssh -XY <username>@fend01.hpc.ku.dk
```

To activate the photpipe environment on the DARK cluster, use the following command:

```
source /lustre/hpc/storage/dark/YSE/photpipe/config/PS1/GPC1v3/GPC1YSE.bash.sourceme
```

This will create a bunch of aliases that can help you navigate around the directories photpipe uses.  The most important are:

```
cdraw # the raw data
cdwork # the directory with intermediate data products
cdconfig # the configuration directory
cdpython # the python codes.  Most YSE-specific scripts are in the YSE subdirectory
cdperl # the perl scripts that manage most of the photpipe stages
```

Similarly, there are some useful environment variables defined:

```
$PIPE_PYTHONSCRIPTS # the python code
$PIPE_DATA # the top-level data directory, containing raw data, workspace, and output lightcurve subdirectories
$PIPE_SRC # the top-level code directory
```

## Add a New Object and Copy Over the Data

Now that we've started up Photpipe, I'll get forced photometry of SN 2021smj, the same object I requested forced photometry for in the other tutorial.  The first step is initializing the photpipe environment and adding the object to YSE's master list.

In [1]:
# basic imports
import os
import numpy as np
import glob

In [2]:
# activates the photpipe environment on DARK
# I suggest adding an alias for this command to your ~/.bashrc file
os.system('source /lustre/hpc/storage/dark/YSE/photpipe/config/PS1/GPC1v3/GPC1YSE.bash.sourceme')

256

In [3]:
cmd = 'YSEmasterlist.py -n 2021smj 12:26:46.56 +08:52:57.66 --type SNIa -z 0.00421'
os.system(cmd)

32512

Now copy the YSE data to directories where photpipe will recognize it.  The `--imsizepix` argument should probably be ~3000 pixels.  Ideally we would use smaller stamps as the point-spread function of Pan-STARRS1 varies really rapidly near the center of the field of view.  But, small stamps sometimes have zeropoint failures because they have too few stars.  So this is a tradeoff - what we really need is a spatially varying PSF photometry scheme or an iterative method that goes back and changes the image stamp sizes for transients with zeropoint failures.

The command below launches in batch mode and I've had some issues with missing data resulting from batch mode here.  This command really needs a validation stage at the end where it ensures that all the data have been copied over and re-submits jobs as needed but I haven't done that yet.

In [4]:
cmd = 'cpYSEdata.py -t 2021smj --imsizepix 3000 --batch'
os.system(cmd)

32512

Once these jobs have finished (`squeue --user=<username>` comes up empty) then we need templates for our object from the PS1 3pi survey.  To do this, we request the templates from IPP with YSEmasterlist.py.  This will take a few minutes, and again, you'll have to check for failures.

In [5]:
cmd = 'YSEmasterlist.py -g --batch -e 2021smj --imsizepix 3000'
os.system(cmd)

32512

If all of the above worked, you should be able to type `cdraw` and see the FITS files in:

```
2021smj/g
2021smj/r
2021smj/i
2021smj/z
2021smj/tmpl/g
2021smj/tmpl/r
2021smj/tmpl/i
2021smj/tmpl/z
```

## Running Photpipe

Once all the data are in place, the next step is to first reduce the template data.  The -condor flag sends everything to the DARK queue system so you don't have to wait interactively for everything to finish

In [6]:
cmd = 'eventloop.pl -eventstmpl 2021smj -condor'
os.system(cmd)

32512

We can check on the status of our jobs with `eventstats.pl` which gives a list of stages and success/failures.  The syntax for `eventstats.pl` is the same as for `eventloop.pl` - just replace eventstats with eventloop in the command:

In [7]:
cmd = 'eventstats.pl -eventstmpl 2021smj -condor'
os.system(cmd)

32512

The output should look something like this:

```
================================================================================
info                           || current/last run  || total
================================================================================
date           stage        CCD|| Nin   Nsuc  Nfail || Nall  Nout (%)      Nfail
2021smj_tmpl_g FINDNEWIM    g  || 1     1     0     || 1     1    (100.0)  0
2021smj_tmpl_g CPFIX        g  || 1     1     0     || 1     1    (100.0)  0
2021smj_tmpl_g DOPHOT       g  || 1     1     0     || 1     1    (100.0)  0
2021smj_tmpl_g ABSPHOT      g  || 1     1     0     || 1     1    (100.0)  0
2021smj_tmpl_r FINDNEWIM    r  || 1     1     0     || 1     1    (100.0)  0
2021smj_tmpl_r CPFIX        r  || 1     1     0     || 1     1    (100.0)  0
2021smj_tmpl_r DOPHOT       r  || 1     1     0     || 1     1    (100.0)  0
2021smj_tmpl_r ABSPHOT      r  || 1     0     1     || 1     0    (  0.0)  1
2021smj_tmpl_i FINDNEWIM    i  || 1     1     0     || 1     1    (100.0)  0
2021smj_tmpl_i CPFIX        i  || 1     1     0     || 1     1    (100.0)  0
2021smj_tmpl_i DOPHOT       i  || 1     1     0     || 1     1    (100.0)  0
2021smj_tmpl_i ABSPHOT      i  || 1     1     0     || 1     1    (100.0)  0
2021smj_tmpl_z FINDNEWIM    z  || 1     1     0     || 1     1    (100.0)  0
2021smj_tmpl_z CPFIX        z  || 1     1     0     || 1     1    (100.0)  0
2021smj_tmpl_z DOPHOT       z  || 1     1     0     || 1     1    (100.0)  0
2021smj_tmpl_z ABSPHOT      z  || 1     1     0     || 1     1    (100.0)  0
2021smj_tmpl_y FINDNEWIM    y  || 0     0     0     || 0     0    (  0.0)  0
2021smj_tmpl_y CPFIX        y  || 0     0     0     || 0     0    (  0.0)  0
2021smj_tmpl_y DOPHOT       y  || 0     0     0     || 0     0    (  0.0)  0
2021smj_tmpl_y ABSPHOT      y  || 0     0     0     || 0     0    (  0.0)  0
================================================================================
```

Here, it looks as though photpipe was unable to measure a zeropoint for the r-band template image.  I'm going to ignore this for now, but in practice you would likely need to re-run cpYSEdata.py with the `--clobber` flag and the argument `--imsizepix 6000`

Once the templates show `1` for all stages in the Nsuc column, you can run the full photometric reduction.  This part will take significantly longer than the templates stage because there are many more images, and photpipe needs to also due difference imaging steps.

In [8]:
cmd = 'eventloop.pl -events 2021smj -condor'
os.system(cmd)

32512

Once these steps have finished - again, use the eventstats.pl command, we need to determine the exact position of our SN so that we can perform forced photometry.  This is done by the `eventrecenter.pl` script, which uses the weighted average of the object detections in difference images to find the best centroid for the transient.  The command is:

In [9]:
cmd = 'eventrecenter.pl -events 2021smj'
os.system(cmd)

32512

If this stage succeeds, then we have a good centroid and can measure photometry at that position across all images.  This is done with a special photpipe stage:

In [10]:
cmd = 'eventloop.pl -events 2021smj -forcestage EVENTLC -condor -redo'
os.system(cmd)

32512

I always use the `-redo` flag with this stage to make sure that if this isn't the first time running the object through photpipe, we don't accidentally produce a lightcurve with only *some* of the photometric epochs.

Once this stage has finished, the output lightcurves are located in `$PIPE_DATA/eventspagesv1/2021smj_tmpl`.  The files you care about are the `*forceddifflc.txt` files, though their format is nearly incomprehensible.  I wrote a script a number of years ago that takes the relevant columns and produces a more condensed photometry file (admittedly in a kind of idiosyncratic format) that contains magnitudes and zeropoint = 27.5 fluxes.  To run:

In [11]:
os.system('mksnanalc.py 2021smj')

32512

If everything went well up to this point, there's a convenience script for reading in those output files and working with the data.

In [12]:
import snana
import matplotlib.pyplot as plt
%matplotlib inline
sn = snana.SuperNova(os.path.expandvars('$PIPE_DATA/eventspagesv1/2021smj_tmpl/GPC1v3_2021smj.snana.dat'))
for filt in np.unique(sn.FLT):
    plt.errorbar(sn.MJD[sn.FLT == filt],sn.MAG[sn.FLT == filt],yerr=sn.MAGERR[sn.FLT == filt],fmt='o')

RuntimeError: $PIPE_DATA/eventspagesv1/2021smj_tmpl/GPC1v3_2021smj.snana.dat does not exist.

## A Few Other Tricks for eventloop.pl

The eventloop.pl script (and eventstats.pl as well) has a ton of options that can be used for debugging or just investigating particular stages of photpipe.  I've tried to list a few below:

```
option               description

-redo                set this flag if you need to repeat the output of a previous stage

-condor              for batch mode.  *don't* use if you want to see the output interactively

-maxin <number>      run only a certain number of images.  Useful for debugging or testing.
    
-stage <stage_name>  only run a certain stage.  If you provide a gibberish name, it will tell you the correct names
    
-forcestage <name>   same as -stage, but for some special stages like EVENTLC above
```

## A Few Notes about Directories

All of the intermediate data products from the stage prior to difference imaging are in `$PIPE_DATA/workspace/<object_name>`, while all of the data products after the difference imaging stage are in `$PIPE_DATA/workspace/<object_name>_tmpl`.  Photometry performed on the images are in `*.dcmp` files, which are a weird format with a FITS header but ASCII catalog data.  I like to read these in with a code I wrote ages ago:

```
from txtobj import txtobj
dcmp = txtobj(my_dcmp_filename,cmpheader=True)
print(dcmp.Xpos,dcmp.Ypos,dcmp.FLUX)

# what's the zeropoint of the fluxes?
from astropy.io import fits
zpt = fits.getval(my_dcmp_filename,'ZPTMAG')
```

## Running Photpipe on the Regular Images

Running photpipe on the regular images for e.g., galaxy photometry is simpler than running the full pipeline.  The pipeline only needs to be run through to the ABSPHOT stage:

In [13]:
cmd = 'eventloop.pl -events 2021smj -stage FINDNEWIM,CPFIX,DOPHOT,ABSPHOT -condor'
os.system(cmd)

32512

These steps will use DOPHOT to measure photometry for all objects in the field, and then you just have to write a quick script to extract the photometry for the objects you care about.  For example:

In [14]:
from txtobj import txtobj
from astropy.io import fits
from astropy.wcs import WCS
dcmp_files = glob.glob(os.path.expandvars('$PIPE_DATA/workspace/2021smj/*/*.dcmp'))
for d in dcmpfiles:
    zpt = fits.getval(d,'ZPTMAG') # if this keyword doesn't exist it may mean that photpipe failed!
    dcmpdata = txtobj(d,cmpheader=True)
    wcs = WCS(d)
    xsn,ysn = wcs.wcs_world2pix(snra,sndec,0)
    sep = np.sqrt((xsn-dcmpdata.Xpos)**2.+(ysn-dcmpdata.Ypos)**2.)
    if len(np.where(sep < 2)[0]):
        mag = -2.5*np.log10(dcmpdata.flux[sep == min(sep)][0])+zpt
        magerr = 1.086*dcmpdata.dflux[sep == min(sep)][0]/dcmpdata.flux[sep == min(sep)][0]
        print(mag,magerr)

NameError: name 'dcmpfiles' is not defined